In [87]:
import numpy as np
import re

In [88]:
f = open('test', 'r')
lines = f.readlines()

In [89]:
lines

['3*x1-2*x2+4*x3+22->max\n',
 'x1+x2+3*x3<=12\n',
 '2*x1+4*x2<=14\n',
 'x1+2*x2<=6\n',
 'x1>=0\n',
 'x2>=0\n',
 'x3>=0']

In [90]:
#Составляем список и множество всех переменных, а также множество переменных, имеющих условие о положительности
var_list = re.findall(r"(?<=x)\d+", lines[0])
all_vars = set(var_list)
pat0 = re.compile(r"x\d+>=0\n*")
pos_vars = set()
pos_cond_line = 0
for i in range(1, len(lines)):
    if pat0.match(lines[i]):
        pos_vars.add(re.search(r'\d+', lines[i]).group())
        if pos_cond_line == 0:
            pos_cond_line = i

In [91]:
pos_vars

{'1', '2', '3'}

In [92]:
all_vars - pos_vars

set()

In [93]:
#Функция замены переменных, не имеющих условий положительности
def rand_var_replace(matr, c, var_to_replace):
    new_var_col = []
    for i in range(matr.shape[0]):
        new_var_col.append([-matr[i][var_to_replace - 1]])
    matr = np.hstack((matr, new_var_col))
    c = np.append(c, -c[var_to_replace - 1])
    return matr, c

In [94]:
#составляем матрицу A и вектора b и c. Также сохраним в переменную свободный коэффициент
#матрица A
A = ['' for _ in range(len(var_list))]

for j in range(1, pos_cond_line):
    #смотрим на условия построчно
    temp = lines[j]
    temp_arr = list()
    for k in var_list:
        #ищем переменную в строке и добавляем в таблицу коэффициент перед ней
        str_search0 = f"-?\d+(?=\*x{k})"
        coef = re.findall(str_search0, temp)
        if coef == []:
            str_search1 = f"-?x{k}"
            coef = re.findall(str_search1, temp)
            if coef == []:
                temp_arr.append(0)
            elif coef[0][0] == "-":
                temp_arr.append(-1)
            else:
                temp_arr.append(1)
        else:
            temp_arr.append(str(coef[0]))
    A = np.vstack([A, temp_arr])
A = np.delete(A, 0, axis = 0)
A = A.astype(np.float64)

#вектор b
b = np.array([])
for l in range(1, pos_cond_line):
    temp = re.findall(r"<=-?\d+|>=-?\d+", lines[l])
    b = np.append(b, temp)

#вектор c
c = np.array(re.findall(r"\d+(?=\*x)", lines[0]))
с = c.astype(str)
c = c.astype(np.float64)

#свободный элемент Q
neg_Q = re.findall(r"[+,-]\d+(?=-)", lines[0])[0]
if neg_Q[0] == '+':
    neg_Q = '-' + neg_Q[1:]
else:
    neg_Q = neg_Q[1:]
neg_Q = float(neg_Q)

In [95]:
#проверка, все ли переменные имеют условие о положительности, и добавление новых переменных с данным условием
neg_replacements = {}
if pos_vars != all_vars:
    neg_vars = all_vars - pos_vars
    for var in neg_vars:
        #создаём словарь, где ключом является номер заменяемой переменной (на его место помещается переменная из замены,
        #идущая с плюсом), а значением - номер, который для простоты присваивается переменной из замены, идущей с плюсом.
        neg_replacements[var] = A.shape[1] + 1
        A, c = rand_var_replace(A, c, int(var))

In [96]:
c

array([3., 2., 4.])

In [97]:
A

array([[1., 1., 3.],
       [2., 4., 0.],
       [1., 2., 0.]])

In [98]:
b

array(['<=12', '<=14', '<=6'], dtype='<U32')

In [99]:
neg_Q

-22.0

Приведение к каноническому виду

In [100]:
#Проверка того, что решается задача минимизации, приведение к ней при необходимости
if lines[0][-4:-1] == 'max':
    c *= -1

In [101]:
#Устранение неравенств, приведение их к виду равенств посредством добавления новых переменных
for i in range(len(b)):
    if "<" in b[i]:
        b[i] = re.findall(r"\d+", b[i])[0]
        A = np.hstack((A, np.zeros((3, 1), dtype="float")))
        A[i][-1] = 1.0
        c = np.append(c, 0.0)
    elif ">" in b[i]:
        b[i] = re.findall(r"\d+", b[i])[0]
        A = np.hstack((A, np.zeros((3, 1), dtype="float")))
        A[i][-1] = -1.0
        c = np.append(c, 0.0)
    else:
        b[i] = re.findall(r"\d+", b[i])[0]
b = b.astype(str)
b = b.astype(float)

[-3. -2. -4.  0.] <
[-3. -2. -4.  0.  0.] <
[-3. -2. -4.  0.  0.  0.] <


In [102]:
A

array([[1., 1., 3., 1., 0., 0.],
       [2., 4., 0., 0., 1., 0.],
       [1., 2., 0., 0., 0., 1.]])

In [103]:
b

array([12., 14.,  6.])

In [104]:
c

array([-3., -2., -4.,  0.,  0.,  0.])